In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta

In [64]:
##Parameters
ticker1 = 'NVDA'
ticker2 = 'GOOGL'
ticker3 = 'AMZN'
start_date = '2020-01-01'
end_date = '2025-05-25'
interval = 'monthly' # 'daily', 'weekly', 'monthly'
invest_amount = 7.5
initial_capital = 300

In [65]:
stock = yf.download('NVDA', start=start_date, end=end_date, multi_level_index=False, progress=False)
stock = stock[['Close']].dropna()

if interval == 'daily':
    invest_dates = stock.index
elif interval == 'weekly':
    invest_dates = stock.resample('W').first().index
elif interval == 'monthly':
    invest_dates = stock.resample('MS').first().index
else:
    raise ValueError("Interval must be 'daily', 'weekly', or 'monthly'.")

dca_log = []

total_units = 0
total_invested = 0

first_date = invest_dates[0]
if first_date in stock.index:
    price = stock.loc[first_date, 'Close']
    units = initial_capital / price
    total_units += units
    total_invested += initial_capital
    dca_log.append({
        'Date': first_date,
        'Price': price,
        'Units Bought': units,
        'Total Units': total_units,
        'Total Invested': total_invested,
        'Value': total_units * price
    })
    
for date in invest_dates[1:]:
    if date in stock.index:
        price = stock.loc[date, 'Close']
        units = invest_amount / price
        total_units += units
        total_invested += invest_amount
        dca_log.append({
            'Date': date,
            'Price': price,
            'Units Bought': units,
            'Total Units': total_units,
            'Total Invested': total_invested,
            'Value': total_units * price
        })


In [67]:
def fetch_data(ticker):
    stock = yf.download(ticker, start=start_date, end=end_date, multi_level_index=False, progress=False)
    stock = stock[['Close']].dropna()

    if interval == 'daily':
        invest_dates = stock.index
    elif interval == 'weekly':
        invest_dates = stock.resample('W').first().index
    elif interval == 'monthly':
        invest_dates = stock.resample('MS').first().index #ME = Month End
    else:
        raise ValueError("Interval must be 'daily', 'weekly', or 'monthly'.")
    
    dca_log = []

    total_units = 0
    total_invested = 0

    first_date = invest_dates[0]
    if first_date in stock.index:
        price = stock.loc[first_date, 'Close']
        units = initial_capital / price
        total_units += units
        total_invested += initial_capital
        dca_log.append({
            'Date': first_date,
            'Price': price,
            'Units Bought': units,
            'Total Units': total_units,
            'Total Invested': total_invested,
            'Value': total_units * price
        })

    for date in invest_dates[1:]:
        if date in stock.index:
            price = stock.loc[date, 'Close']
            units = invest_amount / price
            total_units += units
            total_invested += invest_amount
            dca_log.append({
                'Date': date,
                'Price': price,
                'Units Bought': units,
                'Total Units': total_units,
                'Total Invested': total_invested,
                'Value': total_units * price
            })

    dca_df = pd.DataFrame(dca_log).set_index('Date')
    final_value = dca_df['Value'].iloc[-1]
    gain = final_value - total_invested
    roi = (gain / total_invested) * 100
    print(f"Ticker : {ticker}, Final Value: ${final_value:.2f}, Total Invested: ${total_invested:.2f}, Gain: ${gain:.2f}, ROI: {roi:.2f}%")
    return dca_df


In [68]:
stock1=fetch_data(ticker1)
stock2=fetch_data(ticker2)
stock3=fetch_data(ticker3)

Ticker : NVDA, Final Value: $1747.10, Total Invested: $322.50, Gain: $1424.60, ROI: 441.74%
Ticker : GOOGL, Final Value: $464.69, Total Invested: $322.50, Gain: $142.19, ROI: 44.09%
Ticker : AMZN, Final Value: $429.59, Total Invested: $322.50, Gain: $107.09, ROI: 33.21%


In [69]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=stock1.index, y=stock1['Value'], mode='lines', name=f'{ticker1} Value', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=stock2.index, y=stock2['Value'], mode='lines', name=f'{ticker2} Value', line=dict(color='green')))
fig.add_trace(go.Scatter(x=stock3.index, y=stock3['Value'], mode='lines', name=f'{ticker3} Value', line=dict(color='red')))
fig.add_trace(go.Scatter(x=stock1.index, y=stock1['Total Invested'], mode='lines', name='Total Invested', line=dict(color='orange', dash='dash')))

fig.update_layout(
    title=f'DCA Simulation ({interval})',
    xaxis_title='Date',
    yaxis_title='USD',
    hovermode='x unified',
    template='plotly_white'
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

fig.show()